In [13]:
import io
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [15]:
# Define the path pattern to match all CO files in the directory
path_pattern = '/home/jovyan/Data/daily_pd3/CO_daily_agg3_hour_*.csv'
# CO, IN, ID

file_list = glob.glob(path_pattern)
file_list

['/home/jovyan/Data/daily_pd3/CO_daily_agg3_hour_nov.csv',
 '/home/jovyan/Data/daily_pd3/CO_daily_agg3_hour_dec.csv']

In [16]:
# Initialize an empty list to hold DataFrames
dataframes = []

# Loop through the list of files and read each file into a DataFrame
for file in file_list:
    df = pd.read_csv(file)
    dataframes.append(df)
combined_df = pd.concat(dataframes, ignore_index=True)
combined_df

,no_of_points,no_of_unique_users,grt,day,month,geohash_3
0,253,38,0,1,11,d22
1,1191,197,0,1,11,d35
2,76,11,0,1,11,d2s
3,2,1,0,1,11,d32
4,68,18,0,1,11,d3j
...,...,...,...,...,...,...
26215,147,13,7,31,12,d2s
26216,5659,581,7,31,12,d2e
26217,2,1,7,31,12,d3n
26218,139,6,7,31,12,d2v


In [17]:
combined_df['no_of_unique_users'].min()

1

In [18]:
combined_df['local_date'] = pd.to_datetime(combined_df[['month', 'day']].assign(year=2019).astype(str).apply('-'.join, 1)).dt.strftime('%Y%m%d').astype(int)
combined_df = combined_df.sort_values(by='local_date')
filtered_df = combined_df[combined_df['local_date'] > 20191031]

combined_df

,no_of_points,no_of_unique_users,grt,day,month,geohash_3,local_date
0,253,38,0,1,11,d22,20191101
3120,1505,160,2,1,11,d31,20191101
3121,225,24,2,1,11,d2v,20191101
3122,167916,14971,2,1,11,d2g,20191101
3123,5294,630,2,1,11,d3h,20191101
...,...,...,...,...,...,...,...
17853,5396,572,2,31,12,d3d,20191231
17852,62,8,2,31,12,d4q,20191231
17851,50887,5164,2,31,12,d2g,20191231
17849,3,2,2,31,12,d3p,20191231


# Agg by day

In [21]:
pd_agg_day = combined_df.groupby(['geohash_3', 'local_date']).agg({
    'no_of_unique_users': 'sum',
    'no_of_points': 'sum'
}).reset_index()
pd_agg_day = pd_agg_day[pd_agg_day['no_of_unique_users'] >= 10]
pd_agg_day

,geohash_3,local_date,no_of_unique_users,no_of_points
3,6rf,20191101,85,892
4,6rf,20191102,92,986
5,6rf,20191103,78,808
6,6rf,20191104,83,942
7,6rf,20191105,74,758
...,...,...,...,...
3689,d90,20191227,47,507
3690,d90,20191228,39,317
3691,d90,20191229,41,399
3692,d90,20191230,40,442


In [22]:
pd_agg_day[pd_agg_day['geohash_3'] == 'd90']

,geohash_3,local_date,no_of_unique_users,no_of_points
3633,d90,20191101,84,826
3634,d90,20191102,73,753
3635,d90,20191103,72,730
3636,d90,20191104,74,700
3637,d90,20191105,58,559
...,...,...,...,...
3689,d90,20191227,47,507
3690,d90,20191228,39,317
3691,d90,20191229,41,399
3692,d90,20191230,40,442


In [26]:
# Save the combined DataFrame to a new CSV file
folder_path = '/home/jovyan/Data/daily_pd3/Cleaned/'
pd_agg_day.to_csv(folder_path + 'pd_co_2019_agg3.csv', index=False)

# By 3 hour

In [ ]:
filtered_df = filtered_df.drop(columns=['grt', 'day', 'month'])
filtered_df